In [3]:
import pandas as pd
import numpy as np
import random
import random

In [25]:
# 날짜 설정
dates = pd.date_range(start='2024-01-01', end='2024-01-31')

# 제품 목록 생성
products = ['product_{}'.format(i) for i in range(1, 51)]

# 재고 데이터 생성
data = []
for product in products:
    inventory = random.randint(100, 300)  # 시작 재고를 100~300 사이의 랜덤한 값으로 설정
    max_daily_depletion = max(6, random.randint(0, int(inventory * 0.06)))  # 일별 소진 수량의 최댓값
    min_daily_depletion = max(0, max_daily_depletion - 6)  # 일별 소진 수량의 최솟값
    inventories = []
    for date in dates:
        depletion = random.randint(min_daily_depletion, max_daily_depletion)
        inventory = max(0, inventory - depletion)
        inventories.append(inventory)
    data.append([product] + inventories)


# 데이터 프레임 생성
df = pd.DataFrame(data, columns=['Product'] + [date.strftime('%Y-%m-%d') for date in dates])


# CSV 파일 저장
df.to_csv('inventory.csv', index=False)

df


,Product,2024-01-01,2024-01-02,2024-01-03,2024-01-04,2024-01-05,2024-01-06,2024-01-07,2024-01-08,2024-01-09,...,2024-01-22,2024-01-23,2024-01-24,2024-01-25,2024-01-26,2024-01-27,2024-01-28,2024-01-29,2024-01-30,2024-01-31
0,product_1,130,126,123,121,116,115,108,104,102,...,60,53,46,39,35,30,25,19,18,16
1,product_2,252,250,246,244,240,240,235,230,225,...,189,185,179,173,172,167,164,158,156,151
2,product_3,146,142,137,132,127,125,123,120,117,...,73,69,65,59,53,47,45,41,41,40
3,product_4,267,267,263,260,256,254,253,252,246,...,215,212,211,208,206,202,198,193,190,187
4,product_5,256,250,247,245,240,237,234,229,223,...,184,180,177,175,175,170,164,163,160,160
5,product_6,215,215,215,213,212,207,203,199,194,...,151,148,144,139,137,131,128,122,119,113
6,product_7,209,198,189,178,165,154,146,133,121,...,0,0,0,0,0,0,0,0,0,0
7,product_8,151,145,137,130,124,116,111,109,103,...,41,34,26,23,16,9,5,1,0,0
8,product_9,256,255,255,252,247,244,241,238,236,...,181,177,171,171,166,161,159,157,157,153
9,product_10,124,122,118,114,114,113,107,101,99,...,67,61,58,55,55,53,47,41,39,38


In [9]:
df = pd.read_csv('inventory.csv')


In [28]:
#지수가중이동평균 활용 - 전체 평균을 기반으로, 마지막날에 가까울수록 더 가중치를 두는 모델(EWMA)
def calculate_exponential_moving_average(df, product, span=30):
    df_product = df.loc[df['Product'] == product].iloc[:,1:].T
    df_product.columns = ['Inventory']
    df_product['Inventory'] = df_product['Inventory'].astype(int)
    df_product['daily_usage'] = -df_product['Inventory'].diff()
    df_product['ewma'] = df_product['daily_usage'].ewm(span=span).mean()
    return df_product

products = df['Product'].unique()

for product in products:
    df_product = calculate_exponential_moving_average(df, product)
    
    last_inventory = df_product.iloc[-1]['Inventory']
    last_ewma = df_product.iloc[-1]['ewma']
    
    # 재고 소진 예상일 계산
    expected_days = last_inventory / last_ewma
    
    # 3일 이내에 소진될 것으로 예상되면 알람 발생
    if expected_days <= 3:
        print(f'{product}의 재고가 3일 이내에 소진될 것으로 예상됩니다.')




product_7의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_8의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_14의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_22의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_23의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_25의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_27의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_28의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_35의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_36의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_38의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_40의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_42의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_43의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_44의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_47의 재고가 3일 이내에 소진될 것으로 예상됩니다.


In [31]:
df_product

,Inventory,daily_usage,ewma
2024-01-01,263,NaN,NaN
2024-01-02,261,2.0,2.000000
2024-01-03,259,2.0,2.000000
2024-01-04,256,3.0,2.355794
2024-01-05,252,4.0,2.808833
2024-01-06,252,-0.0,2.169755
2024-01-07,252,-0.0,1.745277
2024-01-08,250,2.0,1.789333
2024-01-09,244,6.0,2.446348
2024-01-10,241,3.0,2.525494


In [21]:
#재고가 0인 제품은 제외

In [29]:
df2 = df[df['2024-01-31'] != 0]
df2

,Product,2024-01-01,2024-01-02,2024-01-03,2024-01-04,2024-01-05,2024-01-06,2024-01-07,2024-01-08,2024-01-09,...,2024-01-22,2024-01-23,2024-01-24,2024-01-25,2024-01-26,2024-01-27,2024-01-28,2024-01-29,2024-01-30,2024-01-31
0,product_1,130,126,123,121,116,115,108,104,102,...,60,53,46,39,35,30,25,19,18,16
1,product_2,252,250,246,244,240,240,235,230,225,...,189,185,179,173,172,167,164,158,156,151
2,product_3,146,142,137,132,127,125,123,120,117,...,73,69,65,59,53,47,45,41,41,40
3,product_4,267,267,263,260,256,254,253,252,246,...,215,212,211,208,206,202,198,193,190,187
4,product_5,256,250,247,245,240,237,234,229,223,...,184,180,177,175,175,170,164,163,160,160
5,product_6,215,215,215,213,212,207,203,199,194,...,151,148,144,139,137,131,128,122,119,113
8,product_9,256,255,255,252,247,244,241,238,236,...,181,177,171,171,166,161,159,157,157,153
9,product_10,124,122,118,114,114,113,107,101,99,...,67,61,58,55,55,53,47,41,39,38
10,product_11,235,232,225,222,214,210,204,201,194,...,126,124,121,114,111,104,97,89,85,79
11,product_12,198,195,190,188,183,181,179,178,178,...,141,141,135,134,128,128,125,122,121,118


In [30]:
# 지수가중이동평균 활용 - 전체 평균을 기반으로, 마지막날에 가까울수록 더 가중치를 두는(EWMA)
def calculate_exponential_moving_average(df2, product, span=30):
    df2_product = df2.loc[df2['Product'] == product].iloc[:,1:].T
    df2_product.columns = ['Inventory']
    df2_product['Inventory'] = df2_product['Inventory'].astype(int)
    df2_product['daily_usage'] = -df2_product['Inventory'].diff()
    df2_product['ewma'] = df2_product['daily_usage'].ewm(span=span).mean()
    return df2_product

products = df2['Product'].unique()

for product in products:
    df2_product = calculate_exponential_moving_average(df2, product)
    
    last_inventory = df2_product.iloc[-1]['Inventory']
    last_ewma = df2_product.iloc[-1]['ewma']
    
    # 재고 소진 예상일 계산
    expected_days = last_inventory / last_ewma
    
    # 3일 이내에 소진될 것으로 예상되면 알람 발생
    if expected_days <= 3:
        print(f'{product}의 재고가 3일 이내에 소진될 것으로 예상됩니다.')




product_14의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_22의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_28의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_35의 재고가 3일 이내에 소진될 것으로 예상됩니다.
product_44의 재고가 3일 이내에 소진될 것으로 예상됩니다.
